Import all relevant packages, as well as local code

In [272]:
import reaktoro as rkt
import thermofun as fun
import numpy as np
from distinctipy import distinctipy
import plotly.graph_objects as go

# local code from ./source
from source.utils import EquilibriumSpecs, EquilibriumConditions

ModuleNotFoundError: No module named 'source.utils'

Create a database object containing the thermodynamic data for each substance

In [27]:
# create ThermoFun database
database = fun.Database("../data/mines19-thermofun.json");

# add Co aqueous species
database.appendData("../data/Co_aq_species_thermo.json");
database.appendData("../data/Co-S_aq_species_thermo_Migdisov_etal_2011_OptimC.json");
# database.appendData("../data/Co-Cl_aq_species_thermo_Liu_etal_2011.json");
database.appendData("../data/Co-Cl_aq_species_thermo_Migdisov_etal_2011_OptimC.json");

# add Co minerals 
database.appendData("../data/Co_mineral_thermo.json");

# convert from ThermoFun to Reaktoro-compatible database
db = rkt.ThermoFunDatabase(database)

## Create a chemical system for each metal-bearing fluid

aqueous species common to each chemical system

In [80]:
general_species = ["H2O@", "H+", "OH-", 
                    "Na+", "NaOH@", "NaCl@", "HCl@", "Cl-", 
                    "H2S@", "HS-", "HSO4-", "SO4-2", 
]

aqueous species for each metal

In [117]:
metals = ["Co", "Cu", "Zn", "Pb"]
metal_species = dict.fromkeys(metals)

metal_species["Co"] = ["Co+2", "Co+3", "CoCl+", "CoCl2@", "CoCl3-", "CoCl4-2", \
                "CoH2S+2", "CoHS+", "CoO2-2", "CoO@", "CoOH+", "CoOH+2", "HCoO2-"
]
metal_species["Cu"] = ["Cu(HS)2-", "Cu(OH)2-", "Cu(OH)@", "Cu+" ,"Cu+2", "CuCl+", "CuCl2-", \
                "CuCl2@", "CuCl3-", "CuCl3-2", "CuCl4-2", "CuCl4-3", "CuCl@", "CuHS@", \
                "CuO2-2", "CuO2H-", "CuO@", "CuOH+"
]
metal_species["Zn"] = ["Zn(HS)2(OH)-", "Zn(HS)2@", "Zn(HS)3-", "Zn(HS)4-2", "Zn+2", "ZnCl+", \
                "ZnCl2@", "ZnCl3-", "ZnO2-2", "ZnO2H-", "ZnO@", "ZnOH+"
]
metal_species["Pb"] = ["Pb(HS)2@", "Pb(HS)3-", "Pb+2", "PbCl+", "PbCl2@", "PbCl3-", \
            "PbCl4-2", "PbF2@", "PbO2H-", "PbO@", "PbOH+"
]

Create a dictionary to store the chemical system (i.e., the collection of relevant mineral, aqueous, and gaseous species) for each fluid

In [273]:
systems = {}

for metal in metals: 
    aqueous_phase = rkt.AqueousPhase(general_species + metal_species[metal])
    aqueous_phase.setActivityModel(rkt.ActivityModelHKF())
    phases = rkt.Phases(db)
    phases.add(aqueous_phase)

    systems[metal] = rkt.ChemicalSystem(phases)


Set model parameters
- temperature
- pressure (assume saturated vapor pressure of water)
- concentration of metal
- concentration of ligands (i.e., Cl and S)

In [274]:
temperature = 200 + 273.15                                     # [K]
pressure = rkt.waterSaturationPressureWagnerPruss(temperature) # [Pa]
metal_concentration = 0.01                                     # [molal]
S_concentration = 0.01                                         # [molal]
Cl_concentration = 0.5                                         # [molal]

$f_{O_2}$ buffered by hematite-magnetite assemblage
$$
 2Fe_3O_4 + 0.5O_2 \leftrightarrow 3Fe_2O_3
$$

In [275]:
def get_fO2(T, P, rxn): 
    lgK = rxn.props(T, P).lgK[0]
    O2_coeff = rxn.equation().coefficients()[1]
    fO2 = 10**((1/O2_coeff)*lgK)

    return fO2

rxn = db.reaction("2*Magnetite + 0.5*O2(g) = 3*Hematite")
fO2 = get_fO2(temperature, pressure, rxn) * 10**10 # [bar]

Set info about metals looped through and how they're added. 

In [276]:
metals = ["Co", "Cu", "Pb", "Zn"]
metal_titrants = ["CoCl2@", "CuCl2@", "PbCl2@", "ZnCl2@"]

In [277]:
species_amounts = {}
for metal in metals: 
    species_amounts[metal] = {}
    for specie in metal_species[metal]: 
        species_amounts[metal][specie] = []

For each metal, loop through a range of pH values and speciate the fluid

In [281]:
for metal, metal_titrant in zip(metals, metal_titrants): 
    # species_list = 
    # get chemical system
    system = systems[metal]

    # create fluid and add water
    fluid = rkt.ChemicalState(system)
    fluid.add("H2O@", 1, "kg")

    # create the constraints for the equilibrium problem at hand
    specs = EquilibriumSpecs(system)
    specs.temperature()
    specs.pressure()
    specs.fugacity("O2(g)")
    specs.pH("HCl@")
    specs.elementConcentration(metal, "molal", metal_titrant)
    specs.elementConcentration("S", "molal", "H2S@")
    specs.elementConcentration("Cl", "molal", "NaCl@")
    specs.electroneutrality("Cl-")

    # create an object to solve the equilibrum problem
    solver = rkt.EquilibriumSolver(specs)

    # loop through pH values
    pH_values = np.linspace(2, 12, 28)
    for pH in pH_values: 
        # set values for each constraint
        conditions = EquilibriumConditions(specs)

        conditions.temperature(temperature, "K")
        conditions.pressure(pressure, "Pa")
        conditions.set("pH", pH)
        conditions.fugacity("O2(g)", fO2, "bar")
        conditions.elementConcentration(metal, metal_concentration)
        conditions.elementConcentration("S", S_concentration)
        conditions.elementConcentration("Cl", Cl_concentration)
        conditions.electroneutrality()


        # compute equilibrium problem and determine if computation was successful
        result = solver.solve(fluid, conditions)
        outcome = result.optima.succeeded
        print(f"converged: {outcome}")


        # get relative amounts of each species
        props = rkt.ChemicalProps(fluid)
        aprops = rkt.AqueousProps(props)
        for specie in species_amounts[metal].keys(): 
            specie_amount = float(aprops.speciesMolality(specie))
            relative_specie_amount = specie_amount / metal_concentration
            species_amounts[metal][specie].append(relative_specie_amount)

converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converged: True
converge

Set plotting color for each species

In [282]:
species_colors = dict.fromkeys(species_amounts.keys())

for metal in metals: 
    species_colors[metal] = dict.fromkeys(species_amounts[metal].keys())
    n = len(species_amounts[metal].keys())
    colors = distinctipy.get_colors(n, rng=2514)
    for color, specie in zip(colors, species_colors[metal].keys()): 
        species_colors[metal][specie] = "rgb" + str(tuple(color))

Plot speciation for each metal! 

In [283]:
for metal in metals: 
    fig = go.Figure()
    for specie in species_amounts[metal]: 
        fig.add_trace(go.Scatter(
            x=pH_values, 
            y=species_amounts[metal][specie], 
            line_color=species_colors[metal][specie], 
            name=specie))

    # format layout
    fig.update_layout(title=f"Speciation of {metal} @ {temperature-273.15} °C, Psat, fO2 = Hem-Mt, [{metal}]={metal_concentration} m, [S]={S_concentration} m, [Cl]={Cl_concentration} m",
                    xaxis_title="pH", 
                    yaxis_title=f"mole proportion of {metal} in species"
    )
    fig.show()